# Traveling Salesman Problem (TSP) - Genetic Algorithm and Greedy Approach

This notebook provides an implementation of both the Genetic Algorithm and the Greedy Algorithm for solving the **Traveling Salesman Problem (TSP)** on multiple datasets. The goal of the TSP is to find the shortest possible route that visits each city exactly once and returns to the origin city.

The Genetic Algorithm is a metaheuristic inspired by the process of natural selection, whereas the Greedy Algorithm is a heuristic that builds a solution step-by-step, choosing the nearest unvisited city at each step.

### Contents
1. [Import Libraries](#Import-Libraries)
2. [Distance Matrix Creation](#Distance-Matrix-Creation)
3. [Genetic Algorithm](#Genetic-Algorithm)
4. [Greedy Algorithm](#Greedy-Algorithm)
5. [Run Both Algorithms on All Datasets](#Run-Both-Algorithms-on-All-Datasets)


In [40]:
## Import Libraries

import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random

logging.basicConfig(level=logging.DEBUG)


## Distance Matrix Creation


In [41]:
def compute_DIST_MATRIX(file_path):
    """
    Creates a distance matrix using geographic coordinates from a CSV file.
    
    Args:
    - file_path (str): Path to the CSV file containing city names, latitudes, and longitudes.
    
    Returns:
    - np.ndarray: Symmetric distance matrix where entry (i, j) represents the distance between city i and city j.
    """
    CITIES = pd.read_csv(file_path, header=None, names=['name', 'lat', 'lon'])
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
    for c1, c2 in combinations(CITIES.itertuples(), 2):
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return DIST_MATRIX


## Genetic Algorithm Parameters and Population Creation


In [42]:
# Parameters for the Genetic Algorithm
POPULATION_SIZE = 100
MAX_GENERATIONS = 1000
OFFSPRING_SIZE = 30

def create_individual(num_cities):
    """
    Creates a random individual (tour) by shuffling city indices.
    """
    return np.random.permutation(num_cities)

def create_population(num_cities):
    """
    Initializes a population of individuals (tours) of a specified size.
    """
    return [create_individual(num_cities) for _ in range(POPULATION_SIZE)]


## Fitness and Distance Calculation


In [43]:
def calculate_distance_path(individual, DIST_MATRIX):
    """
    Calculates the total distance of a tour represented by an individual.
    """
    num_cities = len(individual)
    return sum(DIST_MATRIX[individual[i], individual[i + 1]] for i in range(num_cities - 1)) + DIST_MATRIX[individual[num_cities - 1], individual[0]]

def fitness(individual, DIST_MATRIX):
    """
    Calculates the fitness of an individual as the inverse of the path length (distance).
    """
    return 1 / calculate_distance_path(individual, DIST_MATRIX)


## Crossover and Mutation Operations


In [44]:
def pmx(parent1, parent2, num_cities):
    """
    Performs Partially Mapped Crossover (PMX) to produce a child from two parents.
    """
    start, end = sorted(random.sample(range(num_cities), 2))
    child = [None] * num_cities
    child[start:end] = parent1[start:end]
    for i in range(start, end):
        if parent2[i] not in child:
            index = i
            while start <= index < end:
                index = np.where(parent2 == parent1[index])[0][0]
            child[index] = parent2[i]
    for i in range(num_cities):
        if child[i] is None:
            child[i] = parent2[i]
    return child

def mutate(individual):
    """
    Swaps two random cities in an individual's tour, introducing variation.
    """
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]
    return individual


## Tournament Selection


In [45]:
def tournament_selection(population, fitness_scores, k=3):
    """
    Performs tournament selection to choose an individual for reproduction.
    """
    k = min(k, len(population))
    selected = random.sample(population, k)
    selected_fitness = [fitness_scores[i] for i, ind in enumerate(population) if id(ind) in map(id, selected)]
    best_individual = selected[selected_fitness.index(max(selected_fitness))]
    return best_individual


## Genetic Algorithm Execution


In [46]:
def genetic_algorithm(file_path):
    """
    Executes the Genetic Algorithm to find an optimal TSP solution.
    Tracks the best solution found and its generation.
    """
    DIST_MATRIX = compute_DIST_MATRIX(file_path)
    num_cities = len(DIST_MATRIX)
    population = create_population(num_cities)
    fitness_scores = [fitness(ind, DIST_MATRIX) for ind in population]

    best_individual = None
    best_distance = float('inf')
    best_generation = 0

    for generation in range(MAX_GENERATIONS):
        offspring = []
        for _ in range(OFFSPRING_SIZE):
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            child = pmx(parent1, parent2, num_cities)
            if random.random() < 0.2:
                child = mutate(child)
            offspring.append(child)
        
        population.extend(offspring)
        fitness_scores = [fitness(ind, DIST_MATRIX) for ind in population]
        
        sorted_indices = np.argsort(fitness_scores)[::-1]
        population = [population[i] for i in sorted_indices[:POPULATION_SIZE]]
        fitness_scores = [fitness_scores[i] for i in sorted_indices[:POPULATION_SIZE]]
        
        best_fitness = max(fitness_scores)
        best_individual_current = population[fitness_scores.index(best_fitness)]
        best_distance_current = 1 / best_fitness

        # Update the best solution if the current one is better
        if best_distance_current < best_distance:
            best_distance = best_distance_current
            best_individual = best_individual_current
            best_generation = generation

    return best_individual, best_distance, best_generation


## Greedy Algorithm Execution


In [47]:
def greedy_algorithm(file_path):
    """
    Executes the Greedy Algorithm to find a solution to the TSP.
    Tracks the total cost and the number of steps.
    """
    DIST_MATRIX = compute_DIST_MATRIX(file_path)
    num_cities = len(DIST_MATRIX)

    tour = [0]
    visited = set(tour)
    total_cost = 0
    steps = 0

    while len(tour) < num_cities:
        last = tour[-1]
        next_city = min(
            (i for i in range(num_cities) if i not in visited),
            key=lambda x: DIST_MATRIX[last, x]
        )
        total_cost += DIST_MATRIX[last, next_city]
        tour.append(next_city)
        visited.add(next_city)
        steps += 1

    total_cost += DIST_MATRIX[tour[-1], tour[0]]
    tour.append(tour[0])
    steps += 1  # Final step to return to the starting city

    return tour, total_cost, steps


## Run Both Algorithms on All Datasets


In [48]:
datasets = ["cities/italy.csv", "cities/vanuatu.csv", "cities/russia.csv", "cities/us.csv", "cities/china.csv"]

for dataset in datasets:
    print(f"\nDataset: {dataset}")
    best_individual, best_distance, best_generation = genetic_algorithm(dataset)
    print(f"Genetic Algorithm - Best Solution found at generation {best_generation} with distance {best_distance}")

    _, greedy_cost, greedy_steps = greedy_algorithm(dataset)
    print(f"Greedy Algorithm - Total Cost: {greedy_cost} found in {greedy_steps} steps")

   



Dataset: cities/italy.csv
Genetic Algorithm - Best Solution found at generation 891 with distance 5569.728743725634
Greedy Algorithm - Total Cost: 4436.031769525161 found in 46 steps

Dataset: cities/vanuatu.csv
Genetic Algorithm - Best Solution found at generation 10 with distance 1345.5449564733112
Greedy Algorithm - Total Cost: 1475.528091104531 found in 8 steps

Dataset: cities/russia.csv
Genetic Algorithm - Best Solution found at generation 998 with distance 116137.16419670233
Greedy Algorithm - Total Cost: 42334.16465744784 found in 167 steps

Dataset: cities/us.csv
Genetic Algorithm - Best Solution found at generation 998 with distance 261010.57272560603
Greedy Algorithm - Total Cost: 48050.025864461364 found in 326 steps

Dataset: cities/china.csv
Genetic Algorithm - Best Solution found at generation 999 with distance 572605.1390730924
Greedy Algorithm - Total Cost: 63962.918429455196 found in 726 steps
